In [1]:
import torch
import torch.nn as nn

In [25]:
class SelfAttention(nn.Module):
  def __init__(self, embed_size, heads):
    super(SelfAttention, self).__init__()
    self.embed_size = embed_size
    self.heads = heads
    self.head_dim = embed_size // heads

    assert (self.head_dim * heads == embed_size), "Embed size needs to be div by heads"

    self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.fc_out = nn.Linear(heads*self.head_dim, embed_size)

  def forward(self, values, keys, query, mask):
    N = query.shape[0]
    value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

    # Split embedding into self.heads pieces
    # (N, len, embed_size) -> (N, len, heads, head_dim)
    values = values.reshape(N, value_len, self.heads, self.head_dim)
    keys = keys.reshape(N, key_len, self.heads, self.head_dim)
    queries = query.reshape(N, query_len, self.heads, self.head_dim)

    values = self.values(values)
    keys = self.keys(keys)
    queries = self.queries(queries)

    energy = torch.einsum('nqhd,nkhd->nhqk', [queries, keys]) # (N, heads, query_len, key_len)

    if mask is not None:
      energy = energy.masked_fill(mask == 0, float('-1e20'))

    attention = torch.softmax(energy/(self.embed_size**0.5), dim=3)

    # Attention shape: (N, heads, query_len, key_len)
    # Values shape: (N, value_len, heads, heads_dim)
    # (N, query_len, heads, heads_dim)
    out = torch.einsum('nhql,nlhd->nqhd', [attention, values])
    out = out.reshape(N, query_len, self.heads*self.head_dim)
    out = self.fc_out(out)
    return out

In [26]:
class TransformerBlock(nn.Module):
  def __init__(self, embed_size, heads, dropout, forward_expansion):
    super(TransformerBlock, self).__init__()
    self.attention = SelfAttention(embed_size, heads)
    self.norm1 = nn.LayerNorm(embed_size)
    self.norm2 = nn.LayerNorm(embed_size)

    self.feed_forward = nn.Sequential(
        nn.Linear(embed_size, forward_expansion*embed_size),
        nn.ReLU(),
        nn.Linear(forward_expansion*embed_size, embed_size)
    )
    self.dropout = nn.Dropout(dropout)

  def forward(self, value, key, query, mask):
    attention = self.attention(value, key, query, mask)

    x = self.dropout(self.norm1(attention + query))
    forward = self.feed_forward(x)
    out = self.dropout(self.norm2(forward + x))
    return out

In [27]:
class Encoder(nn.Module):
  def __init__(self,
               src_vocab_size,
               embed_size,
               num_layers,
               heads,
               device,
               forward_expansion,
               dropout,
               max_length
  ):
    super(Encoder, self).__init__()
    self.embed_size = embed_size
    self.device = device
    self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
    self.position_embedding = nn.Embedding(max_length, embed_size)

    self.layers = nn.ModuleList(
        [
            TransformerBlock(
                embed_size,
                heads,
                dropout=dropout,
                forward_expansion=forward_expansion
            )
            for _ in range(num_layers)
        ]
    )
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, mask):
    N, seq_length = x.shape
    positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

    out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

    for layer in self.layers:
      out = layer(out, out, out, mask)

    return out

In [28]:
class DecoderBlock(nn.Module):
  def __init__(self, embed_size, heads, forward_expansion, dropout, device):
    super(DecoderBlock, self).__init__()
    self.attention = SelfAttention(embed_size, heads)
    self.norm = nn.LayerNorm(embed_size)
    self.transformer_block = TransformerBlock(
        embed_size, heads, dropout, forward_expansion
    )
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, value, key, src_mask, trg_mask):
    attention = self.attention(x, x, x, trg_mask)
    query = self.dropout(self.norm(attention+x))
    out = self.transformer_block(value, key, query, src_mask)
    return out

In [29]:
class Decoder(nn.Module):
  def __init__(self,
               trg_vocab_size,
               embed_size,
               num_layers,
               heads,
               forward_expansion,
               dropout,
               device,
               max_length):
    super(Decoder, self).__init__()
    self.device = device
    self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
    self.position_embedding = nn.Embedding(max_length, embed_size)
    self.layers = nn.ModuleList(
        [
            DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
            for _ in range(num_layers)
        ]
    )
    self.fc_out = nn.Linear(embed_size, trg_vocab_size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, enc_out, src_mask, trg_mask):
    N, seq_length = x.shape
    positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
    x = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

    for layer in self.layers:
      x = layer(x, enc_out, enc_out, src_mask, trg_mask)

    out = self.fc_out(x)
    return out

class Transformer(nn.Module):
  def __init__(self,
               src_vocab_size,
               trg_vocab_size,
               src_pad_idx,
               trg_pad_idx,
               embed_size=256,
               num_layers=6,
               forward_expansion=4,
               heads=8,
               dropout=0,
               device='cpu',
               max_length=100
  ):
    super(Transformer, self).__init__()
    self.encoder = Encoder(
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length
    )

    self.decoder = Decoder(
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length
    )

    self.src_pad_idx = src_pad_idx
    self.trg_pad_idx = trg_pad_idx
    self.device=device

  def make_src_mask(self, src):
    src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
    # (N, 1, 1, src_len)
    return src_mask.to(self.device)

  def make_trg_mask(self, trg):
    N, trg_len = trg.shape
    trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
        N, 1, trg_len, trg_len
    )
    return trg_mask.to(self.device)

  def forward(self, src, trg):
    src_mask = self.make_src_mask(src)
    trg_mask = self.make_src_mask(trg)

    enc_src = self.encoder(src, src_mask)
    out = self.decoder(trg, enc_src, src_mask, trg_mask)
    return out

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(device)
trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

src_pad_idx = 0
trg_pad_idx = 0
src_vocab_size = 10
trg_vocab_size = 10
model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx).to(device)
out = model(x, trg[:,:-1])
print(out.shape)

torch.Size([2, 7, 10])


In [31]:
out

tensor([[[-0.8983,  0.7658,  1.0083,  0.3012, -0.7434,  0.6354, -0.0729,
          -0.3913, -0.7436, -1.5433],
         [-0.2656,  0.1374, -0.1788,  0.2311,  0.2549,  0.3430,  0.2369,
           0.9063, -0.7722,  0.0567],
         [-0.9389, -0.0365,  1.3003,  0.3068,  0.1312,  0.4618,  0.0174,
          -0.1831, -0.5926,  0.0258],
         [-0.8179,  0.1909,  0.2076,  0.5756,  0.5377,  0.2627,  0.4117,
           0.9175,  0.3599,  0.1604],
         [-0.7408,  0.9745,  0.9814,  0.1676, -0.2620,  1.2020,  0.3641,
           0.3088, -0.1438, -0.5299],
         [-0.2455,  0.7037,  0.9652,  0.9225,  0.1833,  0.4665,  0.9650,
           0.4382, -0.2079, -0.5700],
         [-0.3213,  0.3019,  0.5731,  0.9077,  0.3636,  0.1180,  0.2583,
           0.7721, -0.1934, -0.3822]],

        [[-0.8569,  0.6146,  1.0492,  0.6053, -0.4283,  0.4605, -0.3821,
          -0.2456, -0.5175, -1.5736],
         [-0.5528,  0.3134,  0.5909,  0.8559,  0.5680,  0.5092, -0.1578,
           1.2725, -0.1694, -0.0604],